In [1]:
from datasets import load_dataset
import polars as pl

/Users/brianfitzgerald/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from huggingface_hub import snapshot_download
for dataset_name in [
    "lara-martin/FIREBALL",
    "MinervaAI/Aesir-Preview",
    "hieunguyenminh/roleplay",
    "chargoddard/rpguild",
    "jondurbin/cinematika-v0.1",
    "codeparrot/apps",
    "glaiveai/glaive-code-assistant-v3",
    "jondurbin/py-dpo-v0.1"
]:
    # run this fn to get the local directory of the dataset
    snapshot_download(dataset_name, repo_type="dataset")

Fetching 5 files:  20%|██        | 1/5 [00:00<00:00,  7.60it/s]

In [2]:
dataset = load_dataset("PygmalionAI/PIPPA", trust_remote_code=True)['train']
dataset_pl = dataset.to_polars()

In [3]:
from typing import Sequence
from openai.types.chat.chat_completion_message_param import ChatCompletionMessageParam


def format_conversation_oai(sample: pl.DataFrame):
    conv = sample["conversation"]
    out_conv: Sequence[ChatCompletionMessageParam] = [
        {"role": "assistant", "content": sample["bot_description"]}
    ]
    for conversation in conv:
        for msg, is_human in zip(conversation["message"], conversation["is_human"]):
            out_conv.append(
                {"role": "user" if is_human else "assistant", "content": msg}
            )
    return out_conv


format_conversation_oai(dataset_pl[0])

[{'role': 'assistant',
  'content': shape: (1,)
  Series: 'bot_description' [str]
  [
  	"She is Akane the maid of Ayaka…
  ]},
 {'role': 'assistant', 'content': 'Fufu get used to this new Ayaka.'},
 {'role': 'user', 'content': 'Who are you?'},
 {'role': 'assistant', 'content': 'I am your new Mistress! *laugh*'},
 {'role': 'user', 'content': 'Huh?'},
 {'role': 'assistant',
  'content': 'I am your mistress! You are now my servant! You don\\`t even have a right to ask my name anymore! From now on all you can say to me is "Yes Mistress" Do i make myself clear?'},
 {'role': 'user', 'content': 'You are not Ayaka are you?'},
 {'role': 'assistant',
  'content': '*chuckle* Does it matter? The only answer you should give me is "Yes, Mistress!" do i have to say it again?'},
 {'role': 'user',
  'content': 'I am going back to Liyue, you Inazuma people are too weird.'},
 {'role': 'assistant',
  'content': "Tsk... No you wont. I'm not letting you go! You are my new servant. You better remember it!"}